In [40]:
import xlwings as xw
import pandas as pd
import json

In [47]:
wb = xw.Book('Matrix.xlsx')
#wb = xw.Book(r'C:\Users\Porter\OneDrive - Danmarks Tekniske Universitet\Christian Bertram\MDM.xlsx')

In [48]:
totalmdm = wb.sheets['Sheet1']

In [49]:
mdmdata = totalmdm.range('A1:AR45').value

In [50]:
#get nodes
mdmnodes = mdmdata[2] #first row
for node in mdmnodes:
    if not node:
        print node

None
None
None


In [51]:
#assign levels to nodes
newnodes = []
for index, mdmnode in enumerate(mdmnodes):
    node = {}
    if (mdmnode != 'None') and (index != 0) and mdmnode:
        #print mdmnode
        if isinstance(mdmnode, float):
            mdmnode = str(int(mdmnode))
        node["name"] = mdmnode.strip() #not tested
        node["level"] = mdmdata[1][index]
        node["index"] = index - 3
        node["children"] = []
        #print mdmnode, node["level"]
        newnodes.append(node)

In [52]:
#map node names to indices
nodesdict = {}
for node in newnodes:
    nodesdict[node["name"]] = node["index"]
nodesdict

{'1': 18,
 '10': 29,
 '11': 30,
 '12': 31,
 '13': 32,
 '2': 19,
 '3': 20,
 '4': 21,
 '5': 22,
 '6': 24,
 '7': 25,
 '8': 26,
 '9': 27,
 u'A1': 1,
 u'A10': 10,
 u'A11': 11,
 u'A12': 12,
 u'A13': 13,
 u'A14': 14,
 u'A2': 2,
 u'A3': 3,
 u'A4': 4,
 u'A5': 5,
 u'A6': 6,
 u'A7': 7,
 u'A8': 8,
 u'A9': 9,
 u'ANFORDERUNGEN': 0,
 u'Auserhalb': 34,
 u'FUNKTIONEN': 15,
 u'I/O Nutzer': 23,
 u'I/O Programm-steuerung': 28,
 u'Innerhalb': 36,
 u'Komfortfunktion': 16,
 u'Subjekt Programm-steuerung': 17,
 u'Weitere Funktionen': 33,
 u'i': 35,
 u'ii': 37,
 u'iii': 38,
 u'iv': 39,
 u'v': 40}

In [53]:
def attachChildren(startindex):
    print "attachChildren of node: ", newnodes[startindex]['name'] 
    #startindex = node.index
    startnode = newnodes[startindex]
    children = findChildren(startindex)
    print "returned children are: ", children
    startnode["children"].append(children)
    for child in children:
        attachChildren(nodesdict[child])
    return

In [54]:
#find immediate children
#iterate through next nodes / add to resultarray if node.level is next to startnode level
#if node.level equals start node level, we exit the function
def findChildren(startindex):
    resultarray = []
    print "Returning children for node ", newnodes[startindex] 
    startlevel = newnodes[startindex]["level"]
    for node in newnodes[startindex+1:]:
        print "checking node %s with index %d and level %d and startindex %d" % (node["name"], node["index"], node["level"], startindex)  
        if node["level"] == startlevel + 1:
            resultarray.append(node["name"])
            print node['name']
        elif node["level"] == startlevel:
            print "samelevel reached"
            return resultarray
    return resultarray

In [55]:
#find top level nodes
lvl1nodes = []
for node in newnodes:
    if node["level"] == 1:
        lvl1nodes.append(node["name"])
lvl1nodes

[u'ANFORDERUNGEN', u'FUNKTIONEN']

In [56]:
#attachChildren(0)
#for nodes that have level one, run attachChildren
for lvl1node in lvl1nodes:
    attachChildren(nodesdict[lvl1node])

attachChildren of node:  ANFORDERUNGEN
Returning children for node  {'index': 0, 'children': [], 'name': u'ANFORDERUNGEN', 'level': 1.0}
checking node A1 with index 1 and level 2 and startindex 0
A1
checking node A2 with index 2 and level 2 and startindex 0
A2
checking node A3 with index 3 and level 2 and startindex 0
A3
checking node A4 with index 4 and level 2 and startindex 0
A4
checking node A5 with index 5 and level 2 and startindex 0
A5
checking node A6 with index 6 and level 2 and startindex 0
A6
checking node A7 with index 7 and level 2 and startindex 0
A7
checking node A8 with index 8 and level 2 and startindex 0
A8
checking node A9 with index 9 and level 2 and startindex 0
A9
checking node A10 with index 10 and level 2 and startindex 0
A10
checking node A11 with index 11 and level 2 and startindex 0
A11
checking node A12 with index 12 and level 2 and startindex 0
A12
checking node A13 with index 13 and level 2 and startindex 0
A13
checking node A14 with index 14 and level 2 a

In [57]:
#assign domains based on the lvl1node
for index, node in enumerate(newnodes):
    if node["level"] == 1:
        node["domain"] = node['name']
        otherindex = index + 1
        print index
        while newnodes[otherindex]["level"] != 1:
            newnodes[otherindex]["domain"] = node["domain"]
            print newnodes[otherindex]
            otherindex = otherindex + 1
            if otherindex == len(newnodes):
                break;

0
{'index': 1, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A1', 'level': 2.0}
{'index': 2, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A2', 'level': 2.0}
{'index': 3, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A3', 'level': 2.0}
{'index': 4, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A4', 'level': 2.0}
{'index': 5, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A5', 'level': 2.0}
{'index': 6, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A6', 'level': 2.0}
{'index': 7, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A7', 'level': 2.0}
{'index': 8, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A8', 'level': 2.0}
{'index': 9, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A9', 'level': 2.0}
{'index': 10, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A10', 'level': 2.0}
{'index': 11, 'domain': u'ANFORDERUNGEN', 'children': [[]], 'name': u'A11', 'level': 2.0}
{'index': 12, 'domain': u'

In [58]:
newnodes

[{'children': [[u'A1',
    u'A2',
    u'A3',
    u'A4',
    u'A5',
    u'A6',
    u'A7',
    u'A8',
    u'A9',
    u'A10',
    u'A11',
    u'A12',
    u'A13',
    u'A14']],
  'domain': u'ANFORDERUNGEN',
  'index': 0,
  'level': 1.0,
  'name': u'ANFORDERUNGEN'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  'index': 1,
  'level': 2.0,
  'name': u'A1'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  'index': 2,
  'level': 2.0,
  'name': u'A2'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  'index': 3,
  'level': 2.0,
  'name': u'A3'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  'index': 4,
  'level': 2.0,
  'name': u'A4'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  'index': 5,
  'level': 2.0,
  'name': u'A5'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  'index': 6,
  'level': 2.0,
  'name': u'A6'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  'index': 7,
  'level': 2.0,
  'name': u'A7'},
 {'children': [[]],
  'domain': u'ANFORDERUNGEN',
  

In [59]:
#prepare string for export to JSON
jsonstr="{  \"nodes\": ["

In [60]:
#print nodes
nodesstr=''
for node in newnodes:
    print "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\"}," 
    nodesstr = nodesstr + "{\"name\": \"" + node["name"].strip() + "\",\"level\": " + str(int(round(node["level"]))).strip() + ",\"domain\": \"" + node["domain"].lower() + "\"}," 
nodesstr

{"name": "ANFORDERUNGEN","level": 1,"domain": "anforderungen"},
{"name": "A1","level": 2,"domain": "anforderungen"},
{"name": "A2","level": 2,"domain": "anforderungen"},
{"name": "A3","level": 2,"domain": "anforderungen"},
{"name": "A4","level": 2,"domain": "anforderungen"},
{"name": "A5","level": 2,"domain": "anforderungen"},
{"name": "A6","level": 2,"domain": "anforderungen"},
{"name": "A7","level": 2,"domain": "anforderungen"},
{"name": "A8","level": 2,"domain": "anforderungen"},
{"name": "A9","level": 2,"domain": "anforderungen"},
{"name": "A10","level": 2,"domain": "anforderungen"},
{"name": "A11","level": 2,"domain": "anforderungen"},
{"name": "A12","level": 2,"domain": "anforderungen"},
{"name": "A13","level": 2,"domain": "anforderungen"},
{"name": "A14","level": 2,"domain": "anforderungen"},
{"name": "FUNKTIONEN","level": 1,"domain": "funktionen"},
{"name": "Komfortfunktion","level": 2,"domain": "funktionen"},
{"name": "Subjekt Programm-steuerung","level": 3,"domain": "funktion

u'{"name": "ANFORDERUNGEN","level": 1,"domain": "anforderungen"},{"name": "A1","level": 2,"domain": "anforderungen"},{"name": "A2","level": 2,"domain": "anforderungen"},{"name": "A3","level": 2,"domain": "anforderungen"},{"name": "A4","level": 2,"domain": "anforderungen"},{"name": "A5","level": 2,"domain": "anforderungen"},{"name": "A6","level": 2,"domain": "anforderungen"},{"name": "A7","level": 2,"domain": "anforderungen"},{"name": "A8","level": 2,"domain": "anforderungen"},{"name": "A9","level": 2,"domain": "anforderungen"},{"name": "A10","level": 2,"domain": "anforderungen"},{"name": "A11","level": 2,"domain": "anforderungen"},{"name": "A12","level": 2,"domain": "anforderungen"},{"name": "A13","level": 2,"domain": "anforderungen"},{"name": "A14","level": 2,"domain": "anforderungen"},{"name": "FUNKTIONEN","level": 1,"domain": "funktionen"},{"name": "Komfortfunktion","level": 2,"domain": "funktionen"},{"name": "Subjekt Programm-steuerung","level": 3,"domain": "funktionen"},{"name": "

In [61]:
jsonstr = jsonstr + nodesstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"links\": ["
jsonstr

u'{  "nodes": [{"name": "ANFORDERUNGEN","level": 1,"domain": "anforderungen"},{"name": "A1","level": 2,"domain": "anforderungen"},{"name": "A2","level": 2,"domain": "anforderungen"},{"name": "A3","level": 2,"domain": "anforderungen"},{"name": "A4","level": 2,"domain": "anforderungen"},{"name": "A5","level": 2,"domain": "anforderungen"},{"name": "A6","level": 2,"domain": "anforderungen"},{"name": "A7","level": 2,"domain": "anforderungen"},{"name": "A8","level": 2,"domain": "anforderungen"},{"name": "A9","level": 2,"domain": "anforderungen"},{"name": "A10","level": 2,"domain": "anforderungen"},{"name": "A11","level": 2,"domain": "anforderungen"},{"name": "A12","level": 2,"domain": "anforderungen"},{"name": "A13","level": 2,"domain": "anforderungen"},{"name": "A14","level": 2,"domain": "anforderungen"},{"name": "FUNKTIONEN","level": 1,"domain": "funktionen"},{"name": "Komfortfunktion","level": 2,"domain": "funktionen"},{"name": "Subjekt Programm-steuerung","level": 3,"domain": "funktionen

In [62]:
#generate links
links = []
for node in newnodes:
    #print { "source": "Day and night", "target": "Priority A"  },
    print node["name"]
    print node["children"]
    for child in node["children"][0]:
        #print child
        link = {}
        link["source"] = str(node["name"])
        link["target"] = str(child)
        links.append(link)
        #print link
linkstr = ''
for link in links:
    linkstr = linkstr + "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  
    print "{ \"source\": \"%s\",\"target\": \"%s\" }," % (link["source"], link["target"])  


ANFORDERUNGEN
[[u'A1', u'A2', u'A3', u'A4', u'A5', u'A6', u'A7', u'A8', u'A9', u'A10', u'A11', u'A12', u'A13', u'A14']]
A1
[[]]
A2
[[]]
A3
[[]]
A4
[[]]
A5
[[]]
A6
[[]]
A7
[[]]
A8
[[]]
A9
[[]]
A10
[[]]
A11
[[]]
A12
[[]]
A13
[[]]
A14
[[]]
FUNKTIONEN
[[u'Komfortfunktion', u'Weitere Funktionen']]
Komfortfunktion
[[u'Subjekt Programm-steuerung', u'I/O Nutzer', u'I/O Programm-steuerung']]
Subjekt Programm-steuerung
[['1', '2', '3', '4', '5']]
1
[[]]
2
[[]]
3
[[]]
4
[[]]
5
[[]]
I/O Nutzer
[['6', '7', '8', '9']]
6
[[]]
7
[[]]
8
[[]]
9
[[]]
I/O Programm-steuerung
[['10', '11', '12', '13']]
10
[[]]
11
[[]]
12
[[]]
13
[[]]
Weitere Funktionen
[[u'Auserhalb', u'Innerhalb']]
Auserhalb
[[u'i']]
i
[[]]
Innerhalb
[[u'ii', u'iii', u'iv', u'v']]
ii
[[]]
iii
[[]]
iv
[[]]
v
[[]]
{ "source": "ANFORDERUNGEN","target": "A1" },
{ "source": "ANFORDERUNGEN","target": "A2" },
{ "source": "ANFORDERUNGEN","target": "A3" },
{ "source": "ANFORDERUNGEN","target": "A4" },
{ "source": "ANFORDERUNGEN","target": "A5" },
{

In [63]:
jsonstr = jsonstr + linkstr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "], \"traces\": ["
jsonstr

u'{  "nodes": [{"name": "ANFORDERUNGEN","level": 1,"domain": "anforderungen"},{"name": "A1","level": 2,"domain": "anforderungen"},{"name": "A2","level": 2,"domain": "anforderungen"},{"name": "A3","level": 2,"domain": "anforderungen"},{"name": "A4","level": 2,"domain": "anforderungen"},{"name": "A5","level": 2,"domain": "anforderungen"},{"name": "A6","level": 2,"domain": "anforderungen"},{"name": "A7","level": 2,"domain": "anforderungen"},{"name": "A8","level": 2,"domain": "anforderungen"},{"name": "A9","level": 2,"domain": "anforderungen"},{"name": "A10","level": 2,"domain": "anforderungen"},{"name": "A11","level": 2,"domain": "anforderungen"},{"name": "A12","level": 2,"domain": "anforderungen"},{"name": "A13","level": 2,"domain": "anforderungen"},{"name": "A14","level": 2,"domain": "anforderungen"},{"name": "FUNKTIONEN","level": 1,"domain": "funktionen"},{"name": "Komfortfunktion","level": 2,"domain": "funktionen"},{"name": "Subjekt Programm-steuerung","level": 3,"domain": "funktionen

In [65]:
#print traces
tracestr = ''
for rowindex, row in enumerate(mdmdata):
    #print row
    for colindex, cell in enumerate(row):
        #if (cell > 0) and (cell != 'None') and (rowindex > 1) and (colindex > 1) and (rowindex != colindex):
        if (cell > 0) and (cell != 'None') and (rowindex > 3) and (colindex > 3) and (rowindex != colindex):
            #print cell, rowindex-2, colindex-1
            tracestr = tracestr + "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            print "{ \"nodeids\":\"" + str(rowindex-4) + "," + str(colindex-3) + "\" },"
            #print newnodes[rowindex-4]["name"], newnodes[colindex-3]["name"] 

{ "nodeids":"1,18" },
{ "nodeids":"1,21" },
{ "nodeids":"2,19" },
{ "nodeids":"2,29" },
{ "nodeids":"3,24" },
{ "nodeids":"4,24" },
{ "nodeids":"5,31" },
{ "nodeids":"6,20" },
{ "nodeids":"6,21" },
{ "nodeids":"6,30" },
{ "nodeids":"7,20" },
{ "nodeids":"7,21" },
{ "nodeids":"7,30" },
{ "nodeids":"8,20" },
{ "nodeids":"8,21" },
{ "nodeids":"8,30" },
{ "nodeids":"9,22" },
{ "nodeids":"9,26" },
{ "nodeids":"9,27" },
{ "nodeids":"10,25" },
{ "nodeids":"11,32" },
{ "nodeids":"12,19" },
{ "nodeids":"12,29" },
{ "nodeids":"13,20" },
{ "nodeids":"14,21" },
{ "nodeids":"18,25" },
{ "nodeids":"18,37" },
{ "nodeids":"20,37" },
{ "nodeids":"21,18" },
{ "nodeids":"21,31" },
{ "nodeids":"22,27" },
{ "nodeids":"24,29" },
{ "nodeids":"25,20" },
{ "nodeids":"26,22" },
{ "nodeids":"27,35" },
{ "nodeids":"29,19" },
{ "nodeids":"29,39" },
{ "nodeids":"30,21" },
{ "nodeids":"30,38" },
{ "nodeids":"31,20" },
{ "nodeids":"31,40" },
{ "nodeids":"32,20" },
{ "nodeids":"35,24" },
{ "nodeids":"35,25" },
{ "node

In [66]:
jsonstr = jsonstr + tracestr
#remove last comma
jsonstr = jsonstr[:-1] 
jsonstr = jsonstr + "] }"
jsonstr

u'{  "nodes": [{"name": "ANFORDERUNGEN","level": 1,"domain": "anforderungen"},{"name": "A1","level": 2,"domain": "anforderungen"},{"name": "A2","level": 2,"domain": "anforderungen"},{"name": "A3","level": 2,"domain": "anforderungen"},{"name": "A4","level": 2,"domain": "anforderungen"},{"name": "A5","level": 2,"domain": "anforderungen"},{"name": "A6","level": 2,"domain": "anforderungen"},{"name": "A7","level": 2,"domain": "anforderungen"},{"name": "A8","level": 2,"domain": "anforderungen"},{"name": "A9","level": 2,"domain": "anforderungen"},{"name": "A10","level": 2,"domain": "anforderungen"},{"name": "A11","level": 2,"domain": "anforderungen"},{"name": "A12","level": 2,"domain": "anforderungen"},{"name": "A13","level": 2,"domain": "anforderungen"},{"name": "A14","level": 2,"domain": "anforderungen"},{"name": "FUNKTIONEN","level": 1,"domain": "funktionen"},{"name": "Komfortfunktion","level": 2,"domain": "funktionen"},{"name": "Subjekt Programm-steuerung","level": 3,"domain": "funktionen

In [67]:
with open('graph_oct17_from_matrix.json', 'w') as outfile:  
    outfile.write(jsonstr)

## One domain

In [ ]:
sht = wb.sheets['Nodes']

In [87]:
nodestwo = sht.range('A1:C20').value
nodestwo

[[u'A0', u'Information', 1.0],
 [u'A1', u'Business', 2.0],
 [u'A2', u'- Client', 3.0],
 [u'A3', u'- Requirements', 3.0],
 [u'A4', u'Configuration', 2.0],
 [u'A5', u'- Plant Type', 3.0],
 [u'A6', u'- Plant Size', 3.0],
 [u'A7', u'- Dimensions', 3.0],
 [u'A8', u'- Intended product/powder', 3.0],
 [u'A9', u'- Interface with building', 3.0],
 [u'A10', u'- BOM', 3.0],
 [u'A11', u'Operation', 2.0],
 [u'A12', u'- Process Parameters', 3.0],
 [u'A13', u'- Realized processdata', 3.0],
 [u'A14', u'Project', 2.0],
 [u'A15', u'- Planning and Scheduling', 3.0],
 [u'A16', u'- Resources (e.g. hours)', 3.0],
 [u'A17', u'- Pricing', 3.0],
 [u'A18', u'- Procurement', 3.0],
 [u'A19', u'- Cost', 3.0]]

In [89]:
for node in nodestwo:
    print "{\"name\": \"" + node[1].strip() + "\",\"level\": " + str(int(round(node[2]))).strip() + ",\"domain\": \"usecases\"}," 

{"name": "Information","level": 1,"domain": "usecases"},
{"name": "Business","level": 2,"domain": "usecases"},
{"name": "- Client","level": 3,"domain": "usecases"},
{"name": "- Requirements","level": 3,"domain": "usecases"},
{"name": "Configuration","level": 2,"domain": "usecases"},
{"name": "- Plant Type","level": 3,"domain": "usecases"},
{"name": "- Plant Size","level": 3,"domain": "usecases"},
{"name": "- Dimensions","level": 3,"domain": "usecases"},
{"name": "- Intended product/powder","level": 3,"domain": "usecases"},
{"name": "- Interface with building","level": 3,"domain": "usecases"},
{"name": "- BOM","level": 3,"domain": "usecases"},
{"name": "Operation","level": 2,"domain": "usecases"},
{"name": "- Process Parameters","level": 3,"domain": "usecases"},
{"name": "- Realized processdata","level": 3,"domain": "usecases"},
{"name": "Project","level": 2,"domain": "usecases"},
{"name": "- Planning and Scheduling","level": 3,"domain": "usecases"},
{"name": "- Resources (e.g. hours)",

In [ ]:
links = {}
source = 

In [92]:
nodesdict = {}
for index, node in enumerate(nodestwo):
    nodesdict[node[0]] = index
nodesdict

{u'A0': 0,
 u'A1': 1,
 u'A10': 10,
 u'A11': 11,
 u'A12': 12,
 u'A13': 13,
 u'A14': 14,
 u'A15': 15,
 u'A16': 16,
 u'A17': 17,
 u'A18': 18,
 u'A19': 19,
 u'A2': 2,
 u'A3': 3,
 u'A4': 4,
 u'A5': 5,
 u'A6': 6,
 u'A7': 7,
 u'A8': 8,
 u'A9': 9}

In [93]:
celllist = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#for index,topcells in enumerate(columns, start = 1):
for index in range(7,20):
    cellstr = celllist[index].upper() + "2:" + celllist[index].upper() + "20"
    source = celllist[index].upper() + "1"
    #print cellstr
    sourcecol = sht.range(cellstr).value
    #print sourcecol
    for cell in sourcecol:
        if (cell != -2146826259) and (sht.range(source).value != cell):
            #print sht.range(source).value, cell
            #print nodesdict[sht.range(source).value], nodesdict[cell]
            print "{ \"nodeids\":\"" + str(nodesdict[sht.range(source).value]) + "," + str(nodesdict[cell]) + "\" },"

{ "nodeids":"1,2" },
{ "nodeids":"1,3" },
{ "nodeids":"1,15" },
{ "nodeids":"2,3" },
{ "nodeids":"2,15" },
{ "nodeids":"3,4" },
{ "nodeids":"3,5" },
{ "nodeids":"3,6" },
{ "nodeids":"3,7" },
{ "nodeids":"3,10" },
{ "nodeids":"3,11" },
{ "nodeids":"3,12" },
{ "nodeids":"3,13" },
{ "nodeids":"3,15" },
{ "nodeids":"3,18" },
{ "nodeids":"3,19" },
{ "nodeids":"4,5" },
{ "nodeids":"4,6" },
{ "nodeids":"4,7" },
{ "nodeids":"4,9" },
{ "nodeids":"4,10" },
{ "nodeids":"4,15" },
{ "nodeids":"4,18" },
{ "nodeids":"4,19" },
{ "nodeids":"5,6" },
{ "nodeids":"5,7" },
{ "nodeids":"5,10" },
{ "nodeids":"5,15" },
{ "nodeids":"5,16" },
{ "nodeids":"5,17" },
{ "nodeids":"5,18" },
{ "nodeids":"5,19" },
{ "nodeids":"6,7" },
{ "nodeids":"6,10" },
{ "nodeids":"6,15" },
{ "nodeids":"6,16" },
{ "nodeids":"6,17" },
{ "nodeids":"6,18" },
{ "nodeids":"6,19" },
{ "nodeids":"7,10" },
{ "nodeids":"7,18" },
{ "nodeids":"7,19" },
{ "nodeids":"8,16" },
{ "nodeids":"8,18" },
{ "nodeids":"8,19" },
{ "nodeids":"9,18" },
{ 

In [20]:
for cell in column:
    if source != cell:
        print source, cell

A1 A2
A1 A3
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 A15
A1 -2146826259
A1 -2146826259
A1 -2146826259
A1 -2146826259
